In [2]:
import pandas as pd
from __future__ import division
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pickle

In [3]:
source_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Yes Bank\\Data\\Featurized\\'
target_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Yes Bank\\Model\\'
#file_name = 'Consolidated_data_set.csv'
file_name = 'YESBANK.csv'
model_name ='Comprehensive_model.sav'
source_file_path = source_folder +  file_name
target_file_path = target_folder + model_name
stock_data = pd.read_csv(source_file_path)

In [4]:
stock_data = stock_data.replace([np.inf, -np.inf], np.nan)

In [5]:
stock_data.fillna(0,inplace=True)

In [6]:
attr_cols = (stock_data.columns).tolist()

In [7]:
feature_cols = []
signal_cols = []
for i in attr_cols:
    attr_list=[]
    attr_list = i.split('_')
    if 'RBC' in attr_list or 'DBC' in attr_list or 'RBCV' in attr_list or 'DBCV' in attr_list  or 'Add' in attr_list:
        feature_cols.append(i)
    if 'Signal' in attr_list and 'V' not in attr_list:
        signal_cols.append(i)

In [8]:
#features = stock_data[['% Change', '% V_Change', 
#        'Roll_Change_30', 'Roll_VChange_30', 'DB_CMax_30', 'DB_CMin_30', 'DB_CMean_30', 'DB_CVMax_30', 'DB_CVMin_30', 'DB_CVMean_30', 
#        'Roll_Change_50', 'Roll_VChange_50', 'DB_CMax_50', 'DB_CMin_50', 'DB_CMean_50', 'DB_CVMax_50', 'DB_CVMin_50', 'DB_CVMean_50', 
#       'Roll_Change_150', 'Roll_VChange_150', 'DB_CMax_150', 'DB_CMin_150', 'DB_CMean_150', 'DB_CVMax_150', 'DB_CVMin_150', 'DB_CVMean_150',
#       'Roll_Change_200', 'Roll_VChange_200', 'DB_CMax_200', 'DB_CMin_200', 'DB_CMean_200', 'DB_CVMax_200', 'DB_CVMin_200', 'DB_CVMean_200']]

In [9]:
#features = stock_data[['% Change',  
#        'Roll_Change_30', 'DB_CMax_30', 'DB_CMin_30', 'DB_CMean_30', 'DB_CVMean_30', 
#        'Roll_Change_50', 'DB_CMax_50', 'DB_CMin_50', 'DB_CMean_50',  'DB_CVMean_50',
#        'Roll_Change_150', 'DB_CMax_150', 'DB_CMin_150', 'DB_CMean_150', 'DB_CVMean_150',
#        'Roll_Change_200', 'DB_CMax_200', 'DB_CMin_200', 'DB_CMean_200', 'DB_CVMean_200']]

In [10]:
signal_cols

['DF_Signal_Buy_10',
 'DF_Signal_Sell_10',
 'DF_Signal_Buy_15',
 'DF_Signal_Sell_15',
 'DF_Signal_Buy_20',
 'DF_Signal_Sell_20',
 'DF_Signal_Buy_25',
 'DF_Signal_Sell_25',
 'DF_Signal_Buy_30',
 'DF_Signal_Sell_30',
 'DF_Signal_Buy_50',
 'DF_Signal_Sell_50',
 'DF_Signal_Buy_150',
 'DF_Signal_Sell_150',
 'DF_Signal_Buy_200',
 'DF_Signal_Sell_200']

In [11]:
features = stock_data[feature_cols]
features = features[200:]

In [12]:
#features.head()

In [13]:
#train_target.isnull().sum()

In [14]:
#test_target.hist()
#plt.show()

In [15]:
#train_data[0:5]

In [16]:
def normalize_data(train_data,test_data):
    from sklearn.preprocessing import normalize
    train_data = normalize(train_data,axis=0)
    #train_target = normalize(train_target)
    test_data = normalize(test_data,axis=0)
    #test_target = normalize(test_target)
    return train_data, test_data

In [17]:
def run_predictions_for_t_column(model_performance,t_signal,first_run,features,train_data,train_target,test_data,test_target):
    from sklearn.metrics import accuracy_score
    import operator
    from sklearn.metrics import confusion_matrix
    import sklearn as sk
    from sklearn.linear_model import LogisticRegression
    features_columns = features.columns
    baseline = len(test_target[test_target == 0])/len(test_target)
    
    for penalty in ('l1','l2'):
        #for maxiter in (1000,2000,3000,4000,5000,10000,50000,100000):
        for maxiter in (4000,5000):
            #for tolerance in (1e+10,1e+8,1e+6,1e+4,1e+2,1e+0,1e-2,1e-4,1e-6,1e-8,1e-10,1e-12):
            for tolerance in (1e-4,1e-6,1e-8,1e-10,1e-12):
                #print ('processing '  +str(t_signal))
                model = sk.linear_model.LogisticRegression(penalty=penalty,max_iter=maxiter,tol=tolerance)
                #model.fit(train_data.as_matrix(),train_target.as_matrix())
                model.fit(train_data,train_target)
                #model.coef_
                #features_columns
                map_coeff_columns = dict()
                for i in range(len(features_columns)):
                    map_coeff_columns[features_columns[i]] = model.coef_[0][i]
                #print (map_coeff_columns)
                sorted_map_coeff_columns = sorted(map_coeff_columns.items(),key=operator.itemgetter(1),reverse=True)
                #sorted_map_coeff_columns
                #accuracy = accuracy_score(y_true=test_target.as_matrix(), y_pred=model.predict(test_data))
                accuracy = accuracy_score(y_true=test_target, y_pred=model.predict(test_data))
                #print ("Test Accuracy: %s" % accuracy)
                #print ("Baseline accuracy (majority class classifier): %s" % baseline)
                predicted_signal = model.predict(test_data)
                cmat = confusion_matrix(y_true=test_target.as_matrix(),
                                        y_pred=model.predict(test_data),
                                        labels=model.classes_)    # use the same order of class as the LR model.
                #print (' target_label | predicted_label | count ')
                #print ('--------------+-----------------+-------')
                # Print out the confusion matrix.
                # NOTE: Your tool may arrange entries in a different order. Consult appropriate manuals.
                false_action = 0
                false_no_action = 0
                true_action = 0
                true_no_action = 0
                for i, target_label in enumerate(model.classes_):
                    for j, predicted_label in enumerate(model.classes_):
                        #print ('{0:^13} | {1:^15} | {2:5d}'.format(target_label, predicted_label, cmat[i,j]))
                        if target_label == predicted_label:
                            if target_label ==0:
                                true_no_action = cmat[i,j]
                            else:
                                true_action = cmat[i,j]
                        elif target_label == 0:
                            false_action = cmat[i,j]
                        else:
                            false_no_action = cmat[i,j]
                        if (true_action + false_action) == 0:
                            precision = 0
                        
                            precision = true_action / (true_action + false_action)
                        recall = true_action/ (true_action + false_no_action)

                # save the model to disk
                model_name = 'Yesbank_model_' +str(t_signal) +str('_') +str(penalty) +str('_') + str (maxiter) +str('_') + str(tolerance) +str('.sav')
                target_file_path = target_folder + model_name
                pickle.dump(model, open(target_file_path, 'wb'))
                new_row = {'Stock':['YesBank'],'Date':['2017-11-21'],'Prediction':t_signal,'Penalty':penalty,'Max_Iter':maxiter,'Tolerance':tolerance,'Model_Coeff': str(sorted_map_coeff_columns),'Accuracy':accuracy,'Majority_Class':baseline,'Confusion_Matrix':str(cmat),'Model_Classes':str(model.classes_),'CM_True_No_Action':true_no_action,'CM_True_Buy/Sell':true_action,'CM_False_Buy/Sell':false_action,'CM_False_Inaction':false_no_action,'Precision':precision,'Recall':recall}

                model_performance = model_performance.append(new_row,ignore_index=True)

                #model_name ='Yesbank_model_OneVsRest_2017-10-26.sav'
                #target_file_path = target_folder + model_name
                #pickle.dump(model_OneVsRest, open(target_file_path, 'wb'))
    return model_performance

In [18]:
#precision = model.evaluate(test_data, metric='precision')['precision']
#print "Precision on test data: %s" % precision

#from sklearn.metrics import precision_score
#precision = precision_score(y_true=test_target.as_matrix(), 
#                            y_pred=model.predict(test_data))
#print ("Precision on test data: %s" % precision)

In [19]:
#recall = model.evaluate(test_data, metric='recall')['recall']
#print "Recall on test data: %s" % recall

#from sklearn.metrics import recall_score
#recall = recall_score(y_true=test_target.as_matrix(),
#                      y_pred=model.predict(test_data))
#print ("Recall on test data: %s" % recall)

In [20]:
#model.predict_log_proba(train_data)

In [21]:
#model.predict_proba(test_data)

In [22]:
#def apply_threshold(probabilities, threshold, predicted_signal):
    ### YOUR CODE GOES HERE
    # +1 if >= threshold and -1 otherwise.
#    prediction = pd.Series(len(probabilities))
#    for i in range(len(probabilities)):
#        if probabilities[i] > threshold:
#            prediction[i] = predicted_signal[i]
#        #elif predicted_signal[i] == 0:
        #    prediction[i] = 0
        #elif predicted_signal[i] == -1:
        #    prediction[i] = -1
#        else:
#            prediction[i] = 2 # unless strong signal, take no position (buy/sell)
#    return prediction

In [23]:
#probabilities = model.predict_proba(test_data)[:,1]
#predictions_with_default_threshold = apply_threshold(probabilities, 0.5, predicted_signal)
#predictions_with_high_threshold = apply_threshold(probabilities, 0.8, predicted_signal)

In [24]:
#from sklearn.metrics import confusion_matrix
#cmat = confusion_matrix(y_true=test_target.as_matrix(),
#                        y_pred=predictions_with_high_threshold,
#                        labels=model.classes_)    # use the same order of class as the LR model.
#print (' target_label | predicted_label | count ')
#print ('--------------+-----------------+-------')
# Print out the confusion matrix.
# NOTE: Your tool may arrange entries in a different order. Consult appropriate manuals.
#for i, target_label in enumerate(model.classes_):
#    for j, predicted_label in enumerate(model.classes_):
#        print ('{0:^13} | {1:^15} | {2:5d}'.format(target_label, predicted_label, cmat[i,j]))

In [25]:
#type(probabilities)

In [26]:
#prob_df = pd.DataFrame(probabilities)

In [27]:
#prob_df[0].hist()
#plt.show()

In [28]:
#prob_df.describe()

In [29]:
#from sklearn.preprocessing import label_binarize
#from sklearn import svm, datasets

# Use label_binarize to be multi-label like settings
#Y = label_binarize(target, classes=[-1, 0, 1])
#n_classes = Y.shape[1]
#X = features

# Split into training and test
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25, random_state=42)

# We use OneVsRestClassifier for multi-label prediction
#from sklearn.multiclass import OneVsRestClassifier

# Run classifier
#classifier = OneVsRestClassifier(svm.LinearSVC(random_state=42))
#model_OneVsRest = classifier.fit(X_train, Y_train)
#y_score = classifier.decision_function(X_test)
#y_pred = classifier.predict(X_test)

In [30]:
#y_score

In [31]:
#y_pred

In [32]:
#from sklearn.metrics import average_precision_score
#from sklearn.metrics import average_recall_score
#average_precision = average_precision_score(Y_test, y_score)
#average_recall = average_recall_score(Y_test,y_score)

#print('Average precision-recall score: {0:0.2f}'.format(average_precision))

In [33]:
#from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import average_precision_score

# For each class
#precision = dict()
#recall = dict()
#average_precision = dict()
#for i in range(n_classes):
#    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
#                                                        y_score[:, i])
#    average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
#precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
#    y_score.ravel())
#average_precision["micro"] = average_precision_score(Y_test, y_score,
#                                                     average="micro")
#print('Average precision score, micro-averaged over all classes: {0:0.2f}'

#      .format(average_precision["micro"]))

In [34]:
#y_score

In [35]:
#Y_test.shape

In [36]:
#Y_test

In [37]:
#match = 0
#minus_1 = 0
#zero = 0
#plus_1 = 0
#unknown = 0
#for i in range(3):
#    for x in range(len(Y_test)):
#        if Y_test[x][i] == y_pred[x][i]:
#            match = match + 1
#        elif y_pred[x][0] == 1:
#            minus_1 = minus_1 + 1
#        elif y_pred[x][1] == 1:
#            zero = zero + 1
#        elif y_pred[x][2] == 1:
#            plus_1 = plus_1 + 1
#        else:
#            unknown = unknown + 1
#    print (str(i) +' matched ' +str(match))
#    print (str(minus_1) +' ' +str(zero) +' ' +str(plus_1) +' ' +str(unknown))
#    match = 0
#    minus_1 = 0
#    zero = 0
#    plus_1 = 0
#    unknown = 0
#print ('total rows ' +str(len(Y_test))) 

In [38]:
#model_read = pickle.load(open(target_file_path, 'rb'))

In [39]:
#model_read

In [40]:
#a = y_pred[0]

In [41]:
#no_pred = 0
#for i in range(len(y_pred)):
#    if y_pred[i].sum() > 1:
#        no_pred += 1
#        print ('no pred ' +str(y_pred[i]))
#print ('no pred ' +str(no_pred))

In [42]:
first_run = True
new_row = {'Stock':['YesBank'],'Date':['2017-11-21'],'Prediction':[0],'Penalty':[0],'Max_Iter':[0],'Tolerance':[0],'Model_Coeff': [0],'Accuracy':[0],'Majority_Class':[0],'Confusion_Matrix':[0],'Model_Classes':[0],'CM_True_No_Action':[0],'CM_True_Buy/Sell':[0],'CM_False_Buy/Sell':[0],'CM_False_Inaction':[0],'Precision':[0],'Recall':[0]}
model_performance = pd.DataFrame(data=new_row,index=None,columns=('Stock','Date','Prediction', 'Penalty','Max_Iter','Tolerance','Model_Coeff','Accuracy','Majority_Class','Confusion_Matrix','CM_True_No_Action','CM_True_Buy/Sell','CM_False_Buy/Sell','CM_False_Inaction','Precision','Recall'))

for t_signal in signal_cols:
    #print (t_signal)
    target = stock_data[t_signal]
    target = target[200:]
    train_data, test_data, train_target, test_target = train_test_split(features, target, test_size=0.25, random_state=42)
    train_data, test_data = normalize_data(train_data,test_data)
    model_performance = run_predictions_for_t_column(model_performance,t_signal,first_run,features,train_data,train_target,test_data,test_target)
    first_run = False  

ZeroDivisionError: division by zero

In [ ]:
file_name = 'Model_Performance_2017-11-21.csv'
target_file_path = target_folder + file_name
model_performance.to_csv(target_file_path)